In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pe
from dimod.reference.samplers import ExactCQMSolver
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel
import time


In [2]:
n_items = 50000
max_cost = 9999
max_weight = 9999

rebuild_ds = False
if rebuild_ds:
    df = pd.DataFrame(
        {
            "cost": np.random.randint(1, max_cost, size=n_items),
            "weight": np.random.randint(1, max_weight, size=n_items),
        }
    )
    df.to_csv("ds.csv", index=False, header=False)


In [3]:
def knapsack_pyomo(values, weights, max_weight, debug=False):
    print(f"solving for {len(values)} items")
    print("build the model")
    pecm = pe.ConcreteModel(name="Knapsack")
    pecm.x = pe.Var(range(0, len(values)), domain=pe.Boolean)
    pecm.worth = pe.Objective(
        expr=sum(values[j] * pecm.x[j] for j in range(0, len(values))),
        sense=pe.maximize,
    )
    pecm.weight = pe.ConstraintList()
    pecm.weight.add(
        sum(weights[j] * pecm.x[j] for j in range(0, len(values))) <= max_weight
    )
    if debug:
        pecm.pprint()

    solver_name = "glpk"
    print(f"submit model to solver {solver_name}")
    solver = pe.SolverFactory(solver_name)
    t_start = time.time()
    solver.solve(pecm)
    t_end = time.time()
    t_solver = t_end - t_start
    if debug:
        pecm.display()

    print("parse the solver output")
    total_value = int(pecm.worth())
    total_weight = int(sum(weights[j] * pecm.x[j]() for j in range(0, len(values))))
    selected_items = pecm.x

    return (
        total_value,
        total_weight,
        selected_items,
        t_solver,
    )


In [4]:
def knapsack_dwave(values, weights, max_weight, debug=False, classic_solver=False):
    print(f"solving for {len(values)} items")
    print("build the model")
    cqm = ConstrainedQuadraticModel()
    obj = BinaryQuadraticModel(vartype="BINARY")
    constraint = QuadraticModel()

    for i in range(len(values)):
        obj.add_variable(i)
        obj.set_linear(i, -values[i])
        constraint.add_variable("BINARY", i)
        constraint.set_linear(i, weights[i])

    cqm.set_objective(obj)
    cqm.add_constraint(constraint, sense="<=", rhs=max_weight, label="capacity")

    if classic_solver:
        sampler = ExactCQMSolver()
    else:
        sampler = LeapHybridCQMSampler()

    if classic_solver:
        print(f"submit model to solver ExactCQMSolver")
    else:
        print(f"submit model to solver {sampler.solver.name}")
    if classic_solver:
        sampleset = sampler.sample_cqm(cqm)
    else:
        sampleset = sampler.sample_cqm(cqm, label="knapsack problem")

    print("parse the solver output")
    feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)
    if not len(feasible_sampleset):
        raise ValueError("No feasible solution found")
    best = feasible_sampleset.first

    selected_items = [key for key, val in best.sample.items() if val == 1.0]
    total_weight = sum(list(weights.loc[selected_items]))
    total_value = sum(list(values.loc[selected_items]))
    best_solution_energy = best.energy

    if classic_solver:
        t_solver_server_side = None
        t_qpu = None
    else:
        t_solver_server_side = sampleset.info["run_time"] / 1000000
        t_qpu = sampleset.info["qpu_access_time"] / 1000000

    return (
        total_value,
        total_weight,
        selected_items,
        t_solver_server_side,
        t_qpu,
        best_solution_energy,
        sampleset,
    )


In [5]:
ds = pd.read_csv("ds.csv", names=["cost", "weight"])

values_array = ds["cost"]
weights_array = ds["weight"]
knapsack_max_weight = int(0.8 * ds["weight"].sum())

finalVal, finalWeight, finalChoices, solver_time = knapsack_pyomo(
    values_array, weights_array, knapsack_max_weight
)

print(f"solver time:         {solver_time}")
print(f"knapsack max weight: {knapsack_max_weight}")
print(f"items total weight:  {finalWeight}")
print(f"items total value:   {finalVal}")


solving for 50000 items
build the model
submit model to solver glpk
parse the solver output
solver time:         129.44616556167603
knapsack max weight: 200052600
items total weight:  200052581
items total value:   241399378


In [6]:
(
    finalVal,
    finalWeight,
    finalChoices,
    solver_time_server,
    solver_time_qpu,
    best_energy,
    sampleset,
) = knapsack_dwave(values_array, weights_array, knapsack_max_weight)

print(f"solver time server side: {solver_time_server}")
print(f"solver time QPU:         {solver_time_qpu}")
print(f"knapsack max weight:     {knapsack_max_weight}")
print(f"items total weight:      {finalWeight}")
print(f"items total value:       {finalVal}")
print(f"best sol. energy:        {best_energy}")


solving for 50000 items
build the model
submit model to solver hybrid_constrained_quadratic_model_version1
parse the solver output
solver time server side: 18.006079
solver time QPU:         0.016048
knapsack max weight:     200052600
items total weight:      200052560
items total value:       240318952
best sol. energy:        -240318952.0


In [7]:
sampleset.info


{'constraint_labels': ['capacity'],
 'qpu_access_time': 16048,
 'charge_time': 8336600,
 'run_time': 18006079,
 'problem_id': 'ac656e5c-7e18-4a95-a554-d386a785950f',
 'problem_label': 'knapsack problem'}